In [1]:
import uxarray as ux
import pymoab
from pymoab import types
from pymoab import topo_util
from pymoab import core
import numpy as np

/uxarray-moab/uxarray/io/_fesom2.py:95: SyntaxWarning: invalid escape sequence '\s'
  sep="\s+",
/uxarray-moab/uxarray/io/_fesom2.py:126: SyntaxWarning: invalid escape sequence '\s'
  sep="\s+",
/uxarray-moab/uxarray/io/_fesom2.py:156: SyntaxWarning: invalid escape sequence '\s'
  sep="\s+",
/uxarray-moab/uxarray/io/_fesom2.py:187: SyntaxWarning: invalid escape sequence '\s'
  sep="\s+",


In [2]:
ls

CITATION.cff        LICENSE      build/     pyproject.toml  uxarray.egg-info/
CODE_OF_CONDUCT.md  MANIFEST.in  build.sh*  setup.py
CONTRIBUTING.md     README.md    ci/        test/
INSTALLATION.md     benchmarks/  docs/      uxarray/


In [ ]:
h5mfile = "../../test/meshfiles/moab/poly2000.h5m"

/moab/MeshFiles/examples/earth


/venv/lib/python3.12/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [ ]:
# Initialize MOAB core
mb = core.Core()

# Load the mesh file
mb.load_file(h5mfile)

# Get all vertices
vert_category = types.MBVERTEX
vertices = mb.get_entities_by_type(0, vert_category)

# Get vertex coordinates
coords = mb.get_coords(vertices)

# Get elements
elements = mb.get_entities_by_dimension(0, 2) 

# Get connectivity for each polygon
connectivity = [mb.get_connectivity(e) for e in elements]

# Convert to a NumPy object array (since polygons may have varying vertex counts)
connectivity_np = np.array(connectivity, dtype=object)

# Print results
print("Vertices:\n", coords.reshape(-1, 3))  # Assuming 3D coordinates
print("Connectivity:\n", connectivity_np)

Vertices:
 [[ 0.03535845 -0.3273244   0.94425024]
 [-0.55521551  0.76218733 -0.33287567]
 [-0.80685849 -0.47905493  0.3456671 ]
 ...
 [-0.50203534 -0.0950418  -0.85960897]
 [ 0.31495228  0.09129587 -0.94470637]
 [ 0.32767601  0.05787308 -0.94301598]]
Connectivity:
 [array([4005, 4135, 4598, 5394, 5082], dtype=uint64)
 array([4149, 4314, 4750, 5651, 5053], dtype=uint64)
 array([2342, 2465, 3573, 2876, 3112], dtype=uint64) ...
 array([2015, 2756, 3729, 3875, 2109, 2221, 3876], dtype=uint64)
 array([4155, 4927, 5939, 5148, 4731, 5940, 4928], dtype=uint64)
 array([4245, 4650, 4257, 5981, 5325, 5801, 5982], dtype=uint64)]


In [22]:
elements = mb.get_entities_by_dimension(0, 2)
elements

	Polygon 1 - 115
	Polygon 3277 - 5058
	Polygon 6007 - 6109

In [30]:
import uxarray as ux
print(coords.shape)
print(connectivity_np.shape)

(5996, 3)
(2000,)


In [45]:
import uxarray as ux
import numpy as np
import matplotlib.pyplot as plt
from pymoab import core, types

def cartesian_to_spherical(xyz):
    x, y, z = xyz[:, 0], xyz[:, 1], xyz[:, 2]
    r = np.sqrt(x**2 + y**2 + z**2)
    lat = np.arcsin(z / r)
    lon = np.arctan2(y, x)
    return np.degrees(lon), np.degrees(lat)

# Initialize MOAB core
mb = core.Core()

# Load the mesh file
filename = "poly2000.h5m"
mb.load_file(filename)

# Get all vertices
vertices = mb.get_entities_by_type(0, types.MBVERTEX)
coords = mb.get_coords(vertices).reshape(-1, 3)  # Assuming 3D coordinates

# Get 2D polygonal elements
elements = mb.get_entities_by_dimension(0, 2)

# Get connectivity for each polygon
connectivity = [mb.get_connectivity(e) for e in elements]

# Ensure all indices are 0-based if necessary (adjust for 1-based indexing)
connectivity = [[idx - 1 for idx in face] for face in connectivity]

# Convert to spherical coordinates (longitude, latitude)
lon, lat = cartesian_to_spherical(coords)

# Convert connectivity list to numpy array, ensuring it is the correct shape
max_face_nodes = max(len(face) for face in connectivity)
connectivity_np = np.full((len(connectivity), max_face_nodes), 0, dtype=np.uint64)

# Populate the array
for i, face in enumerate(connectivity):
    connectivity_np[i, :len(face)] = face

# Create a Ugrid object
grid = ux.Grid.from_topology(node_lon=lon, node_lat=lat, face_node_connectivity=connectivity_np, fill_value=0)

# Plot the grid
grid.plot()

:Path   [Longitude,Latitude]

In [41]:
max_face_nodes

7

In [42]:
connectivity_np

array([[4004, 4134, 4597, ..., 5081,    0,    0],
       [4148, 4313, 4749, ..., 5052,    0,    0],
       [2341, 2464, 3572, ..., 3111,    0,    0],
       ...,
       [2014, 2755, 3728, ..., 2108, 2220, 3875],
       [4154, 4926, 5938, ..., 4730, 5939, 4927],
       [4244, 4649, 4256, ..., 5324, 5800, 5981]], dtype=uint64)